In [1]:
import pandas as pd
from scipy import stats
import numpy as np

In [2]:
%store -r survey_scores
%store -r force_sensitive_stopping_task_ssrt
%store -r go_task_accuracy_before_stop_onset
%store -r go_task_accuracy_after_stop_onset
%store -r duration_of_inhibition
%store -r ssrt_first_half
%store -r ssrt_second_half
%store -r simple_stop_ssrt

### Convert all values from seconds to milliseconds

In [3]:
# Convert to ms
force_sensitive_stopping_task_ssrt.loc[:, 'non_ai'] *= 1000
force_sensitive_stopping_task_ssrt.loc[:, 'ai_disengaged'] *= 1000
force_sensitive_stopping_task_ssrt.loc[:, 'ai_engaged'] *= 1000

duration_of_inhibition.loc[:, 'non_ai'] *= 1000
duration_of_inhibition.loc[:, 'ai_disengaged'] *= 1000
duration_of_inhibition.loc[:, 'ai_engaged'] *= 1000

ssrt_first_half.loc[:, 'non_ai'] *= 1000
ssrt_first_half.loc[:, 'ai_disengaged'] *= 1000
ssrt_first_half.loc[:, 'ai_engaged'] *= 1000

ssrt_second_half.loc[:, 'non_ai'] *= 1000
ssrt_second_half.loc[:, 'ai_disengaged'] *= 1000
ssrt_second_half.loc[:, 'ai_engaged'] *= 1000

### Remove the Mean and SD rows for Simple Stop

In [4]:
simple_stop_ssrt = simple_stop_ssrt.iloc[:-2]

### Rename the the index column to subject ID

In [5]:
force_sensitive_stopping_task_ssrt = force_sensitive_stopping_task_ssrt.rename_axis('subject_id')
simple_stop_ssrt = simple_stop_ssrt.rename_axis('subject_id')
duration_of_inhibition = duration_of_inhibition.rename_axis('subject_id')


### Merge Simple Stop and Force Sensitive Stopping Task SSRTs

In [6]:
merged_df = pd.merge(force_sensitive_stopping_task_ssrt, simple_stop_ssrt[['ssrt', 'ssrt_without_short_ssd_trials', 'ssrt_without_short_ssd_subs']], 
                    left_index=True, right_index=True, how='left')
merged_df.rename(columns={
    'ssrt': 'simple_stop_ssrt',
    'ssrt_without_short_ssd_trials': 'simple_stop_ssrt_without_short_ssd_trials',
    'ssrt_without_short_ssd_subs': 'simple_stop_ssrt_without_short_ssd_subs'
}, inplace=True)

%store merged_df
%store duration_of_inhibition

Stored 'merged_df' (DataFrame)
Stored 'duration_of_inhibition' (DataFrame)


## Stats Functions

In [7]:
def cohens_d_paired(x1, x2):
    """Calculate Cohen's d for paired samples"""
    d = (x1 - x2).mean() / np.sqrt(((x1 - x2).std(ddof=1) ** 2) / 2)
    return d

def calculate_ci_for_difference(x1, x2, confidence=0.95):
    """Calculate confidence interval for the mean difference between two paired samples"""
    diff = x1 - x2
    n = len(diff)
    mean_diff = np.mean(diff)
    sem = stats.sem(diff)  # Standard error of the mean
    ci = stats.t.interval(confidence, n-1, loc=mean_diff, scale=sem)
    return mean_diff, ci

In [8]:
def calc_stats_ind(x1, x2):
    """Calculate t-stat, p-value, Cohen's d, DF, 95% CI for two independent samples"""
    t_stat, p_value = stats.ttest_ind(x1, x2, nan_policy='omit')
    print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.4f}")

    mean_x1 = np.mean(x1)
    mean_x2 = np.mean(x2)
    std_x1 = np.std(x1, ddof=1)  # Sample standard deviation
    std_x2 = np.std(x2, ddof=1)

    # Pooled standard deviation
    n1 = len(x1)
    n2 = len(x2)
    pooled_std = np.sqrt(((n1 - 1) * std_x1**2 + (n2 - 1) * std_x2**2) / (n1 + n2 - 2))

    # Cohen's d
    cohens_d = (mean_x1 - mean_x2) / pooled_std
    print(f"Cohen's d: {cohens_d:.3f}")

    # Degrees of freedom
    degrees_of_freedom = n1 + n2 - 2
    print(f"Degrees of Freedom: {degrees_of_freedom}")

    # Confidence Interval Calculation
    mean_diff = mean_x1 - mean_x2
    se_diff = np.sqrt((std_x1**2 / n1) + (std_x2**2 / n2))

    # 95% CI
    ci = stats.t.interval(0.95, df=degrees_of_freedom, loc=mean_diff, scale=se_diff)
    print(f"95% CI for mean difference: [{ci[0]:.3f}, {ci[1]:.3f}]")

## Planned Statistical Tests

### AI-Disengaged vs Non-AI SSRT

In [9]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai']):.2f}')
print(f'Mean AI-disengaged SSRT: {np.mean(merged_df['ai_disengaged']):.2f}')
ai_disengaged_vs_non_ai = stats.ttest_rel(merged_df['ai_disengaged'], merged_df['non_ai'])
print(f"T-statistic: {ai_disengaged_vs_non_ai.statistic:.2f}, p-value: {ai_disengaged_vs_non_ai.pvalue:.2f}")

Mean Non-AI SSRT: 277.33
Mean AI-disengaged SSRT: 299.47
T-statistic: 5.95, p-value: 0.00


In [10]:
cohens_d = cohens_d_paired(merged_df['ai_disengaged'], merged_df['non_ai'])
mean_diff, ci = calculate_ci_for_difference(merged_df['ai_disengaged'], merged_df['non_ai'])
print(f"Cohen's d: {cohens_d:.2f}")
print(f"Mean difference = {mean_diff:.2f} ms")
print(f"95% CI: [{ci[0]:.2f}, {ci[1]:.2f}] ms")

Cohen's d: 1.33
Mean difference = 22.14 ms
95% CI: [14.62, 29.66] ms


### AI-Engaged vs AI-Disengaged SSRT

In [11]:
print(f'Mean AI-Engaged SSRT: {np.mean(merged_df['ai_engaged']):.2f}')
print(f'Mean AI-Disengaged SSRT: {np.mean(merged_df['ai_disengaged']):.2f}')
ai_engaged_vs_disengaged = stats.ttest_rel(merged_df['ai_engaged'], merged_df['ai_disengaged'])
print(f"T-statistic: {ai_engaged_vs_disengaged.statistic:.2f}, p-value: {ai_engaged_vs_disengaged.pvalue:.2f}")

Mean AI-Engaged SSRT: 332.06
Mean AI-Disengaged SSRT: 299.47
T-statistic: 3.76, p-value: 0.00


In [12]:
cohens_d = cohens_d_paired(merged_df['ai_engaged'], merged_df['ai_disengaged'])
mean_diff, ci = calculate_ci_for_difference(merged_df['ai_engaged'], merged_df['ai_disengaged'])
print(f"Cohen's d: {cohens_d:.2f}")
print(f"Mean difference = {mean_diff:.2f} ms")
print(f"95% CI: [{ci[0]:.2f}, {ci[1]:.2f}] ms")

Cohen's d: 0.84
Mean difference = 32.59 ms
95% CI: [15.08, 50.09] ms


### Non-AI SSRT vs Simple Stop SSRT

In [13]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai']):.2f}')
print(f'Simple Stop SSRT: {np.mean(merged_df['simple_stop_ssrt']):.2f}')
non_ai_vs_simple = stats.ttest_rel(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
non_ai_vs_simple_corr = np.corrcoef(merged_df['non_ai'], merged_df['simple_stop_ssrt'])[1][0]
print(f"T-statistic: {non_ai_vs_simple.statistic:.2f}, p-value = {non_ai_vs_simple.pvalue:.2f}")
print(f"Correlation: {non_ai_vs_simple_corr:.2f}")

Mean Non-AI SSRT: 277.33
Simple Stop SSRT: 210.57
T-statistic: 13.03, p-value = 0.00
Correlation: 0.46


In [14]:
cohens_d2 = cohens_d_paired(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
mean_diff2, ci2 = calculate_ci_for_difference(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
print(f"Cohen's d: {cohens_d2:.2f}")
print(f"Mean difference = {mean_diff2:.2f} ms")
print(f"95% CI: [{ci2[0]:.2f}, {ci2[1]:.2f}] ms")

Cohen's d: 2.91
Mean difference = 66.77 ms
95% CI: [56.40, 77.13] ms


## Order Effects

### Calculate order effects between subjects who had the AI block first vs. the Non-AI block first

In [15]:
# Initialize subjects who had the Non-AI and AI Blocks first
non_ai_first_subs = ['s004', 's009', 's008', 's011', 's012', 's015', 's016', 's019', 's020', 's023', 's024', 's027', 's028', 's031', 's032', 's035', 's036', 's039', 's040', 's043']
ai_first_subs = ['s005', 's006', 's007', 's010', 's013', 's014', 's017', 's018', 's021', 's022', 's025', 's026', 's029', 's030', 's033', 's034', 's037', 's038', 's041', 's042']

In [16]:
ssrt_non_ai_first_non_ai = []
ssrt_ai_first_non_ai = []
ssrt_non_ai_first_ai_disengaged = []
ssrt_ai_first_ai_disengaged = []
ssrt_non_ai_first_ai_engaged = []
ssrt_ai_first_ai_engaged = []

for sub in non_ai_first_subs:
    ssrt_non_ai_first_non_ai.append(force_sensitive_stopping_task_ssrt.loc[sub, "non_ai"])
    ssrt_non_ai_first_ai_disengaged.append(force_sensitive_stopping_task_ssrt.loc[sub, "ai_disengaged"])
    ssrt_non_ai_first_ai_engaged.append(force_sensitive_stopping_task_ssrt.loc[sub, "ai_engaged"])

for sub in ai_first_subs:
    ssrt_ai_first_non_ai.append(force_sensitive_stopping_task_ssrt.loc[sub, "non_ai"])
    ssrt_ai_first_ai_disengaged.append(force_sensitive_stopping_task_ssrt.loc[sub, "ai_disengaged"])
    ssrt_ai_first_ai_engaged.append(force_sensitive_stopping_task_ssrt.loc[sub, "ai_engaged"])

# Calculate the differences between AI SSRT and Non-AI SSRT for both subject groups
diff_non_ai_first = np.array(ssrt_non_ai_first_ai_disengaged) - np.array(ssrt_non_ai_first_non_ai)  # AI-Disengaged - Non-AI for Non-AI first subjects
diff_ai_first = np.array(ssrt_ai_first_ai_disengaged) - np.array(ssrt_ai_first_non_ai)  # AI-Disengaged - Non-AI for AI first subjects

calc_stats_ind(diff_ai_first, diff_non_ai_first)
mean_ai_first = diff_ai_first.mean()
mean_non_ai_first = diff_non_ai_first.mean()

ci_ai_first = stats.t.interval(.95, len(diff_ai_first)-1, loc=mean_ai_first, scale=stats.sem(diff_ai_first))
print(f"Mean difference (AI first): {mean_ai_first:.2f} ms, 95% CI: [{ci_ai_first[0]:.2f}, {ci_ai_first[1]:.2f}] ms")

ci_non_ai_first = stats.t.interval(.95, len(diff_non_ai_first)-1, loc=mean_non_ai_first, scale=stats.sem(diff_non_ai_first))
print(f"Mean difference (Non-AI first): {mean_non_ai_first:.2f} ms, 95% CI: [{ci_non_ai_first[0]:.2f}, {ci_non_ai_first[1]:.2f}] ms")

T-statistic: 1.05, P-value: 0.2988
Cohen's d: 0.333
Degrees of Freedom: 38
95% CI for mean difference: [-7.209, 22.852]
Mean difference (AI first): 26.05 ms, 95% CI: [14.07, 38.03] ms
Mean difference (Non-AI first): 18.23 ms, 95% CI: [8.33, 28.12] ms


### Calculate AI-Disengaged vs. Non-AI SSRT depending on whether they occurred in the first half or second half of the block

#### First half

In [17]:
# T test for AI vs Non AI SSRT in the first half of trials
ai_vs_non_ai = stats.ttest_rel(ssrt_first_half['ai_disengaged'], ssrt_first_half['non_ai'])
print(f"T-statistic: {ai_vs_non_ai.statistic:.2f}, p-value: {ai_vs_non_ai.pvalue:.3f}")

cohens_d1 = cohens_d_paired(ssrt_first_half['ai_disengaged'], ssrt_first_half['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(ssrt_first_half['ai_disengaged'], ssrt_first_half['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference: {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")

print(f'Mean Non-AI SSRT First Half: {np.mean(ssrt_first_half['non_ai']):.2f}')
print(f'Mean AI-Disengaged SSRT First Half: {np.mean(ssrt_first_half['ai_disengaged']):.2f}')

T-statistic: 3.46, p-value: 0.001
Cohen's d: 0.77
Mean difference: 21.40 ms
95% CI: [8.87, 33.92] ms
Mean Non-AI SSRT First Half: 274.94
Mean AI-Disengaged SSRT First Half: 296.33


#### Second Half

In [18]:
ai_vs_non_ai = stats.ttest_rel(ssrt_second_half['ai_disengaged'], ssrt_second_half['non_ai'])
print(f"T-statistic: {ai_vs_non_ai.statistic:.2f}, p-value: {ai_vs_non_ai.pvalue:.3f}")

cohens_d1 = cohens_d_paired(ssrt_second_half['ai_disengaged'], ssrt_second_half['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(ssrt_second_half['ai_disengaged'], ssrt_second_half['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")

print(f'Mean Non-AI SSRT Second Half: {np.mean(ssrt_second_half['non_ai']):.2f}')
print(f'Mean AI-Disengaged SSRT Second Half: {np.mean(ssrt_second_half['ai_disengaged']):.2f}')

T-statistic: 4.62, p-value: 0.000
Cohen's d: 1.03
Mean difference = 24.30 ms
95% CI: [13.66, 34.94] ms
Mean Non-AI SSRT Second Half: 279.72
Mean AI-Disengaged SSRT Second Half: 304.03


## Exploratory T-tests and Confidence Intervals

### Non-AI vs AI-Disengaged Duration of Inhibition

In [19]:
t_stat, p_value = stats.ttest_rel(duration_of_inhibition['ai_disengaged'], duration_of_inhibition['non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.2f}")

cohens_d1 = cohens_d_paired(duration_of_inhibition['ai_disengaged'], 
                            duration_of_inhibition['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(duration_of_inhibition['ai_disengaged'], 
                                              duration_of_inhibition['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference: {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")

print(f'Mean Non-AI Duration of Inhibition: {np.mean(duration_of_inhibition['non_ai']):.2f}')
print(f'Mean AI-Disengaged Duration of Inhibition: {np.mean(duration_of_inhibition['ai_disengaged']):.2f}')

T-statistic: 2.74, P-value: 0.01
Cohen's d: 0.61
Mean difference: 5.74 ms
95% CI: [1.50, 9.98] ms
Mean Non-AI Duration of Inhibition: 41.02
Mean AI-Disengaged Duration of Inhibition: 46.76


### Non-AI vs AI-Disengaged Go Task Accuracy Before Stop Onset

In [20]:
mean_non_ai = go_task_accuracy_before_stop_onset['non_ai'].mean()
mean_ai_disengaged = go_task_accuracy_before_stop_onset['ai_disengaged'].mean()

print(f"Mean Non-AI Accuracy: {mean_non_ai:.2f}")
print(f"Mean AI-Disengaged Accuracy: {mean_ai_disengaged:.2f}")

t_stat, p_value = stats.ttest_rel(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

cohens_d1 = cohens_d_paired(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")


Mean Non-AI Accuracy: 0.87
Mean AI-Disengaged Accuracy: 0.90
T-statistic: 2.81, P-value: 0.008
Cohen's d: 0.63
Mean difference = 0.03 ms
95% CI: [0.01, 0.06] ms


### Non-AI vs AI-Disengaged Go Task Accuracy After Stop Onset (aka Stop Success Rate)

In [21]:
mean_non_ai = go_task_accuracy_before_stop_onset['non_ai'].mean()
mean_ai_disengaged = go_task_accuracy_before_stop_onset['ai_disengaged'].mean()

print(f"Mean Non-AI Accuracy: {mean_non_ai:.2f}")
print(f"Mean AI-Disengaged Accuracy: {mean_ai_disengaged:.2f}")

t_stat, p_value = stats.ttest_rel(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

cohens_d1 = cohens_d_paired(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(go_task_accuracy_before_stop_onset['ai_disengaged'], go_task_accuracy_before_stop_onset['non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f}")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}]")

Mean Non-AI Accuracy: 0.87
Mean AI-Disengaged Accuracy: 0.90
T-statistic: 2.81, P-value: 0.008
Cohen's d: 0.63
Mean difference = 0.03
95% CI: [0.01, 0.06]


### Non-AI vs AI-Disengaged SSRT in subjects who did not show proactive slowing

In [22]:
# Identify non-proactive slowing subjects
non_proactive_slowing_mask = (go_task_accuracy_before_stop_onset['non_ai'] > 
                               go_task_accuracy_before_stop_onset['ai_disengaged'])
non_proactive_slowing_subs = go_task_accuracy_before_stop_onset.index[non_proactive_slowing_mask].tolist()
ssrt_non_ai = force_sensitive_stopping_task_ssrt['non_ai']
ssrt_ai_disengaged = force_sensitive_stopping_task_ssrt['ai_disengaged']

ssrt_data = pd.DataFrame({
    'subject_id': force_sensitive_stopping_task_ssrt.index,
    'ssrt_non_ai': ssrt_non_ai,
    'ssrt_ai_disengaged': ssrt_ai_disengaged
})

ssrt_data = ssrt_data[ssrt_data['subject_id'] != 'mean'] #remove mean row

ssrt_data['difference_ms'] = ssrt_data['ssrt_ai_disengaged'] - ssrt_data['ssrt_non_ai']  # find ai-disengaged - non ai ssrt
ssrt_data['slowing_type'] = ['Non-Proactive' if subject in non_proactive_slowing_subs else 'Proactive' 
                              for subject in ssrt_data['subject_id']]

non_proactive_slowing_data = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']

In [23]:
t_stat, p_value = stats.ttest_rel(non_proactive_slowing_data['ssrt_ai_disengaged'], non_proactive_slowing_data['ssrt_non_ai'])

print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}")

cohens_d1 = cohens_d_paired(non_proactive_slowing_data['ssrt_ai_disengaged'], non_proactive_slowing_data['ssrt_non_ai'])
mean_diff1, ci1 = calculate_ci_for_difference(non_proactive_slowing_data['ssrt_ai_disengaged'], non_proactive_slowing_data['ssrt_non_ai'])
print(f"Cohen's d: {cohens_d1:.2f}")
print(f"Mean difference = {mean_diff1:.2f} ms")
print(f"95% CI: [{ci1[0]:.2f}, {ci1[1]:.2f}] ms")

print(f'Mean Non-AI SSRT: {np.mean(non_proactive_slowing_data['ssrt_non_ai']):.2f}')
print(f'Mean AI-Disengaged SSRT: {np.mean(non_proactive_slowing_data['ssrt_ai_disengaged']):.2f}')

T-statistic: 2.85, P-value: 0.019
Cohen's d: 1.28
Mean difference = 29.13 ms
95% CI: [6.03, 52.24] ms
Mean Non-AI SSRT: 268.66
Mean AI-Disengaged SSRT: 297.79


### T-test comparing AI-Disengaged minus Non-AI SSRT in subjects who proactively slowed and did not proactively slow

In [24]:
non_proactive_differences = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']['difference_ms'].dropna()
proactive_differences = ssrt_data[ssrt_data['slowing_type'] == 'Proactive']['difference_ms'].dropna()

calc_stats_ind(non_proactive_differences, proactive_differences)
mean_non_proactive = ssrt_data[ssrt_data['slowing_type'] == 'Non-Proactive']['difference_ms'].mean()
mean_proactive = ssrt_data[ssrt_data['slowing_type'] == 'Proactive']['difference_ms'].mean()

print(f"Mean difference (Non-Proactive): {mean_non_proactive:.2f} ms")
print(f"Mean difference (Proactive): {mean_proactive:.2f} ms")

T-statistic: 1.09, P-value: 0.2831
Cohen's d: 0.398
Degrees of Freedom: 38
95% CI for mean difference: [-12.624, 31.276]
Mean difference (Non-Proactive): 29.13 ms
Mean difference (Proactive): 19.81 ms


## Correlate survey scores with SSRT

In [25]:
avg_df = merged_df.merge(survey_scores, on='subject_id', how='left')
avg_df['difference_ai_disengaged_and_non_ai_ssrt'] = avg_df['ai_disengaged'] - avg_df['non_ai']
correlation, pval = stats.pearsonr(avg_df['average_score'], avg_df["difference_ai_disengaged_and_non_ai_ssrt"])
print(f"Correlation and p-value between SSRT and survey scores: {correlation:.2f}, {pval:.3f}")

Correlation and p-value between SSRT and survey scores: -0.05, 0.744


## Violations of Context Independence in Simple Stop Data

### a) Non-AI SSRT vs Simple Stop SSRT Excluding trials with SSD < 200 ms

In [26]:
print(f'Mean Non-AI SSRT: {np.mean(merged_df['non_ai']):.2f}')
print(f'Simple Stop SSRT Excluding Trials with SSD < 200ms: {np.mean(merged_df['simple_stop_ssrt_without_short_ssd_trials']):.2f}')
non_ai_vs_simple = stats.ttest_rel(merged_df['non_ai'], merged_df['simple_stop_ssrt'])
print(f"T-statistic: {non_ai_vs_simple.statistic:.2f}, p-value = {non_ai_vs_simple.pvalue:.3f}")

cohens_d2 = cohens_d_paired(merged_df['non_ai'], merged_df['simple_stop_ssrt_without_short_ssd_trials'])
mean_diff2, ci2 = calculate_ci_for_difference(merged_df['non_ai'], merged_df['simple_stop_ssrt_without_short_ssd_trials'])
print(f"Cohen's d: {cohens_d2:.2f}")
print(f"Mean difference = {mean_diff2:.2f} ms")
print(f"95% CI: [{ci2[0]:.2f}, {ci2[1]:.2f}] ms")

Mean Non-AI SSRT: 277.33
Simple Stop SSRT Excluding Trials with SSD < 200ms: 207.72
T-statistic: 13.03, p-value = 0.000
Cohen's d: 2.78
Mean difference = 69.61 ms
95% CI: [58.29, 80.93] ms


### b) Non-AI SSRT vs Simple Stop SSRT Excluding subjects with average SSDs < 200ms

In [27]:
# Remove rows with NaN in simple_stop_ssrt_without_short_ssd_subs
filtered_df = merged_df.dropna(subset=['simple_stop_ssrt_without_short_ssd_subs'])

print(f'Mean Non-AI SSRT: {np.mean(filtered_df['non_ai']):.2f}')
print(f'Simple Stop SSRT Excluding Subjects with SSD < 200ms: {np.mean(filtered_df['simple_stop_ssrt_without_short_ssd_subs']):.2f}')
non_ai_vs_simple = stats.ttest_rel(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
print(f"T-statistic: {non_ai_vs_simple.statistic:.2f}, p-value = {non_ai_vs_simple.pvalue:.3f}")

cohens_d2 = cohens_d_paired(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
mean_diff2, ci2 = calculate_ci_for_difference(filtered_df['non_ai'], filtered_df['simple_stop_ssrt_without_short_ssd_subs'])
print(f"Cohen's d: {cohens_d2:.2f}")
print(f"Mean difference = {mean_diff2:.2f} ms")
print(f"95% CI: [{ci2[0]:.2f}, {ci2[1]:.2f}] ms")

Mean Non-AI SSRT: 280.18
Simple Stop SSRT Excluding Subjects with SSD < 200ms: 208.59
T-statistic: 13.51, p-value = 0.000
Cohen's d: 3.23
Mean difference = 71.59 ms
95% CI: [60.83, 82.36] ms
